In [206]:
import pickle
import torch
import torch.optim as optim
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import data_loader
import models
import pandas as pd

ncsr = pd.read_csv('../justage_vars_init.csv', index_col=0)
pkl = open('../ncsr_for_ddp.pickle', "rb")
dpp_data = {}
dpp_data['ncsr'] = pickle.load(pkl)
ddp_data = dpp_data
pkl.close()

max_len = max([len(ddp_data['ncsr'][x]) for  x in range(len(ddp_data['ncsr']))])
n_event_type = dim_process = len(ncsr.columns) 
n_sample = len(ddp_data['ncsr'])
context_dim = 1


train_input = data_loader.process_seq(ddp_data, list(range(n_sample)), max_len=max_len, n_event_type=n_event_type, tag_batch = 'ncsr', dtype=np.float32)

batch_input_np = list(train_input)
df_patient_static_mat = np.ones((1, n_sample)).astype('float32')
batch_input_np.append(df_patient_static_mat)


gap = batch_input_np[0][:-1, :] - batch_input_np[0][1:, :]
gap_mean = np.mean(gap)
gap_std = np.std(gap)

ddp_model = pickle.load(open('tst.pkl', 'rb'))

In [339]:
max_len = max([len(ddp_data['ncsr'][x]) for  x in range(len(ddp_data['ncsr']))])
n_event_type = dim_process = len(ncsr.columns) 
n_sample = len(ddp_data['ncsr'])
context_dim = 1
test_data = {}
test_data['ncsr'] = []
test_data['ncsr'].append(ddp_data['ncsr'][2][0:9])


tst = data_loader.process_seq(test_data, list(range(1)), max_len=max_len, n_event_type=n_event_type, tag_batch = 'ncsr', dtype=np.float32)
batch_input_np = list(tst)

n_record = tst[0].shape[1]

df_patient_static_mat = np.ones((1, n_sample)).astype('float32')
batch_input_np.append(df_patient_static_mat)
tst = batch_input_np
idx = np.random.choice(list(range(n_record)), size=1, replace=False)
mini_batch = list(map(lambda x: torch.tensor(x[..., idx]), tst))

In [340]:
mini_batch[8]

tensor([[1.]])

In [341]:
with torch.no_grad():
            ddp_model.set_input(*mini_batch)
        
            x = ddp_model.get_prediction()
            y = ddp_model.next_event_time(100)

In [342]:
blank_output.append([y[0], y[1], [x[0] for x in y[2].tolist()]])

ddp_model.get_prediction() dim: 309x126x2

get_prediction:  symptom -> timepoint -> case


In [349]:
blank_output[0]

[tensor([0.1016]),
 tensor([219]),
 [0.0017784187803044915,
  0.006235933862626553,
  0.0026135004591196775,
  0.006934584584087133,
  0.0016796574927866459,
  0.0017474814085289836,
  0.0004928898997604847,
  0.0017940286779776216,
  0.0010215453803539276,
  0.005255051422864199,
  0.0020795834716409445,
  0.0006186982500366867,
  0.0007695105741731822,
  0.0021040805149823427,
  0.002127746120095253,
  0.001978684216737747,
  0.0014316182350739837,
  0.0016522550722584128,
  0.0022313487716019154,
  0.0021269116550683975,
  0.00200055749155581,
  0.0011130566708743572,
  0.002219418529421091,
  0.0017088154563680291,
  0.0017099367687478662,
  0.002056656638160348,
  0.0016049763653427362,
  0.002009954769164324,
  0.0023538183886557817,
  0.0018740472150966525,
  0.003752991557121277,
  0.0025217270012944937,
  0.0013435488799586892,
  0.00411572027951479,
  0.0035502964165061712,
  0.002185362856835127,
  0.002117177238687873,
  0.0015056919073686004,
  0.002742935437709093,
  0.00

In [174]:
for idx, x in enumerate(case_batch):
    case_batch[idx][x!=0]  =  0

In [175]:
case_batch[8][0] = torch.Tensor([np.float32(1.0)]).type(torch.FloatTensor)

In [227]:
type(case_batch)

list